# Marketing Spend

In [0]:
CREATE OR REPLACE VIEW dev.bi.s12n_marketing_spend_mgtv AS 
(
WITH promoted_product_data AS (

--------------------------------------------------------
--------- Collect Non-Affiliate Channel Spend ----------
--------- Paid Search and Paid Social Channels  --------
--------------------------------------------------------
SELECT    
    
    DATE_TRUNC('MONTH', ac.spend_date)::DATE AS spend_month,
    
--     CASE WHEN ac.paid_media_channel IN ('Google Ads','Microsoft Ads') THEN 'Paid Search'
--          ELSE 'Paid Social'
--          END AS l1_channel,
    
    CASE WHEN ac.paid_media_channel ILIKE '%youtube%' THEN 'YouTube Ads'
         ELSE ac.paid_media_channel 
         END AS l2_channel,
    
    CASE WHEN LENGTH(SPLIT_PART(ac.campaign_name, '_', 3)) = 0 OR SPLIT_PART(ac.campaign_name, '_', 3) IS NULL
             THEN 'No Product Identified'
         WHEN SPLIT_PART(ac.campaign_name, '_', 6) ILIKE '%courseraplus%' 
             THEN 'Coursera Plus'
         WHEN SPLIT_PART(ac.campaign_name, '_', 3) IN (SELECT DISTINCT underlying_product_slug FROM prod.gold.underlying_products_vw)
             THEN SPLIT_PART(ac.campaign_name, '_', 3)
         ELSE 'Campaign Name Not Standardized'
         END AS product,
    
    SUM(ac.spend_usd) AS spend

FROM prod.gold_base.paid_media_channel_spend_consolidated ac

WHERE
    -- ac.spend_date BETWEEN '2024-01-01' AND (CURRENT_DATE-2) --DATE RANGE #1
     ac.paid_media_channel IN ('Google Ads','Microsoft Ads','Facebook Ads','Reddit Ads')
     and to_date(ac.spend_date) < to_date(date_trunc('month',current_date))
     and to_date(ac.spend_date) >=to_date(date_trunc('month', add_months(current_date, -2)))

GROUP BY 1,2,3
----------------------------------------------------
--------- Collect Affiliate Channel Spend ----------
----------------------------------------------------

UNION ALL

SELECT 

    DATE_TRUNC('MONTH',conversion_timestamp)::DATE AS spend_month,

--     'Affiliates' AS l1_channel,

    'Affiliates' AS l2_channel,

    CASE WHEN SPLIT_PART(SPLIT_PART(ac.landing_url, '?', 1), '/', 4) = 'courseraplus' 
              THEN 'Coursera Plus' 
         WHEN SPLIT_PART(SPLIT_PART(landing_url, '?', 1), '/', 5) IN (SELECT DISTINCT underlying_product_slug FROM prod.gold.underlying_products_vw) THEN SPLIT_PART(SPLIT_PART(landing_url, '?', 1), '/', 5)
         ELSE 'No Product Identified'
         END AS product,

    SUM(action_cost) as spend

FROM prod.gold_base.affiliate_performance ac

--tables to collect region information
LEFT JOIN prod.gold_base.users u -- for some reason users_vw does not have the country_cd field.
    ON ac.user_id = u.user_id
LEFT JOIN prod.silver.static_countries_vw c
    ON u.country_cd = c.country_cd

--tables to collect transactions information    
LEFT JOIN prod.gold.transactions_vw t
    ON ac.user_id = t.user_id
    AND ac.cart_id = t.cart_id
LEFT JOIN prod.gold.underlying_products_vw up
    ON t.underlying_product_item_id = up.underlying_product_item_id
    AND t.underlying_product_type = up.underlying_product_type

-- WHERE ac.conversion_timestamp::DATE BETWEEN '2024-01-01' AND (CURRENT_DATE-2) --DATE RANGE #2
     where to_date(ac.conversion_timestamp) < to_date(date_trunc('month',current_date))
     and to_date(ac.conversion_timestamp) >=to_date(date_trunc('month', add_months(current_date, -2)))

GROUP BY 1,2,3
),

product_marketing_spend AS (
    SELECT 
        ppd.spend_month,
        up.underlying_product_name,
        up.underlying_product_item_id,
        up.underlying_product_type,
        ppd.product AS campaign_product,
        SUM(ppd.spend) AS spend
    FROM promoted_product_data ppd
    LEFT JOIN prod.gold.underlying_products_vw up
        ON ppd.product = up.underlying_product_slug
    WHERE 
        underlying_product_name IS NOT NULL
    GROUP BY 1,2,3,4,5
)

-- Add base S12N data
SELECT 
    pm.spend_month,
    ps.base_phoenix_specialization_id,
    ps.phoenix_specialization_id,
    DATE_TRUNC('month', sic.phoenix_specialization_display_page_launch_ts)::DATE AS launch_month,
    pm.underlying_product_item_id,
    pm.underlying_product_type,
    pm.campaign_product,
    COALESCE(pm.spend, 0) AS marketing_spend
FROM 
    prod.gold.phoenix_specializations_vw ps
LEFT JOIN
    product_marketing_spend pm
ON 
    ps.phoenix_specialization_id = pm.underlying_product_item_id
JOIN
    dev.bi.s12n_item_counts_mgtv sic -- to get base S12N launch dates
ON 
    ps.base_phoenix_specialization_id = sic.base_phoenix_specialization_id
WHERE spend_month is not null
--     ps.base_phoenix_specialization_id <> ps.phoenix_specialization_id 
-- added the above where clause to check if the join using the phoenix_specialization_id to the underlying_product_item_id is the correct way to do things
)


# Deleting last 2 months of data

In [0]:
DELETE FROM dev.bi.s12n_marketing_spend_mgtf
-- where spend_month is null
WHERE spend_month >= to_date(date_trunc('month', add_months(current_date, -2)))

num_affected_rows
6430


# Appending the latest last 2 months of data

In [0]:
INSERT INTO dev.bi.s12n_marketing_spend_mgtf
select * from dev.bi.s12n_marketing_spend_mgtv

num_affected_rows,num_inserted_rows
25320,25320


# QA